In [7]:
pip install streamlit pandas numpy folium streamlit-folium plotly pydeck requests pyngrok -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.0/524.0 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 100.4 MB/s eta 0:00:00


In [1]:


!cp "/content/drive/MyDrive/assignments_log.csv" /mnt/data/assignments_log.csv

cp: cannot stat '/content/drive/MyDrive/assignments_log.csv': No such file or directory


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!cp "/content/drive/MyDrive/assignments_log.csv" /mnt/data/assignments_log.csv

cp: cannot create regular file '/mnt/data/assignments_log.csv': No such file or directory


In [11]:
# streamlit_app.py
%%writefile streamlit_app.py
import os
import time
import datetime
from math import radians, cos, sin, asin, sqrt

import pandas as pd
import numpy as np
import requests

import streamlit as st
import pydeck as pdk
import plotly.express as px
import folium
from streamlit_folium import st_folium

# ----------------- TomTom & OpenWeather helper code (adapted from your snippets) -----------------

# TOMTOM config (reads from env)
TOMTOM_API_KEY = os.environ.get("TOMTOM_API_KEY", "8tnvwDhRN7nFEDHgC78NtgP1bmQye9vp")
if not TOMTOM_API_KEY or TOMTOM_API_KEY.startswith("<"):
    # we won't hard-fail at import time; the fetch function will raise if key missing for live mode
    TOMTOM_API_KEY = TOMTOM_API_KEY  # keep the provided default

TOMTOM_FLOW_URL = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"


def get_tomtom_speed_kmh(lat, lon, retries=2, pause_sec=0.5):
    params = {"point": f"{lat},{lon}", "key": TOMTOM_API_KEY}
    attempt = 0
    while True:
        try:
            resp = requests.get(TOMTOM_FLOW_URL, params=params, timeout=15)
            if resp.status_code in (429, 500, 502, 503, 504):
                if attempt < retries:
                    attempt += 1
                    time.sleep(pause_sec * (attempt + 1))
                    continue
                resp.raise_for_status()
            resp.raise_for_status()
            data = resp.json()
            fsd = (data or {}).get("flowSegmentData", {}) or {}
            cur = fsd.get("currentSpeed")
            if isinstance(cur, (int, float)) and cur > 0:
                return float(cur)
            ffs = fsd.get("freeFlowSpeed")
            if isinstance(ffs, (int, float)) and ffs > 0:
                return float(ffs)
            return 30.0
        except requests.RequestException:
            if attempt < retries:
                attempt += 1
                time.sleep(pause_sec * (attempt + 1))
                continue
            return 30.0


def batch_get_tomtom_speeds(coords, pause_sec=0.1):
    """
    coords: list of (lat, lon)
    Returns: dict mapping (lat, lon) -> speed_kmh
    """
    rounded = [(round(float(lat), 5), round(float(lon), 5)) for (lat, lon) in coords]
    unique = list(dict.fromkeys(rounded))

    speed_map = {}
    for (rlat, rlon) in unique:
        # if no API key available, simulate
        if not TOMTOM_API_KEY:
            speed_map[(rlat, rlon)] = max(5.0, float(np.random.normal(30, 6)))
        else:
            speed_map[(rlat, rlon)] = get_tomtom_speed_kmh(rlat, rlon)
        time.sleep(pause_sec)

    result = {}
    for (lat, lon), (rlat, rlon) in zip(coords, rounded):
        result[(float(lat), float(lon))] = speed_map[(rlat, rlon)]
    return result


# OPENWEATHER config (reads from env)
OPENWEATHER_API_KEY = os.environ.get("OPENWEATHER_API_KEY", "596eadfb2fd863480d863b7062bd2e8f")
if not OPENWEATHER_API_KEY or OPENWEATHER_API_KEY.startswith("<") or len(OPENWEATHER_API_KEY.strip()) == 0:
    OPENWEATHER_API_KEY = OPENWEATHER_API_KEY  # keep the provided default

OPENWEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"


def get_openweather(lat, lon, retries=2, pause_sec=0.5):
    """
    Query OpenWeather current weather for (lat, lon).
    Returns: dict with description, temp_c, humidity, wind_speed_ms, precip_mm_hr
    If API key missing, returns simulated values.
    """
    if not OPENWEATHER_API_KEY:
        # simulate
        desc = "clear sky"
        return {
            "description": desc,
            "temp_c": float(np.random.normal(25, 4)),
            "humidity": int(np.clip(np.random.normal(60, 10), 10, 100)),
            "wind_speed_ms": float(np.abs(np.random.normal(2, 1))),
            "precip_mm_hr": float(max(0.0, np.random.exponential(0.05))),
        }

    params = {
        "lat": float(lat),
        "lon": float(lon),
        "appid": OPENWEATHER_API_KEY,
        "units": "metric",
    }
    attempt = 0
    while True:
        try:
            resp = requests.get(OPENWEATHER_URL, params=params, timeout=15)
            if resp.status_code in (429, 500, 502, 503, 504):
                if attempt < retries:
                    attempt += 1
                    time.sleep(pause_sec * (attempt + 1))
                    continue
                resp.raise_for_status()
            resp.raise_for_status()
            data = resp.json() or {}

            weather_list = data.get("weather") or []
            description = (weather_list[0].get("description") if weather_list else "") or ""
            main = data.get("main") or {}
            wind = data.get("wind") or {}
            rain = data.get("rain") or {}
            snow = data.get("snow") or {}

            temp_c = float(main.get("temp")) if isinstance(main.get("temp"), (int, float)) else np.nan
            humidity = int(main.get("humidity")) if isinstance(main.get("humidity"), (int, float)) else np.nan
            wind_ms = float(wind.get("speed")) if isinstance(wind.get("speed"), (int, float)) else np.nan

            precip_1h = np.nan
            if isinstance(rain.get("1h"), (int, float)):
                precip_1h = float(rain["1h"])
            elif isinstance(snow.get("1h"), (int, float)):
                precip_1h = float(snow["1h"])

            return {
                "description": str(description).lower(),
                "temp_c": temp_c,
                "humidity": humidity,
                "wind_speed_ms": wind_ms,
                "precip_mm_hr": precip_1h if not np.isnan(precip_1h) else 0.0,
            }
        except requests.RequestException:
            if attempt < retries:
                attempt += 1
                time.sleep(pause_sec * (attempt + 1))
                continue
            return {
                "description": "",
                "temp_c": np.nan,
                "humidity": np.nan,
                "wind_speed_ms": np.nan,
                "precip_mm_hr": 0.0,
            }


def batch_get_openweather(coords, pause_sec=0.1):
    rounded = [(round(float(lat), 5), round(float(lon), 5)) for (lat, lon) in coords]
    unique = list(dict.fromkeys(rounded))

    weather_map = {}
    for (rlat, rlon) in unique:
        weather_map[(rlat, rlon)] = get_openweather(rlat, rlon)
        time.sleep(pause_sec)

    result = {}
    for (lat, lon), (rlat, rlon) in zip(coords, rounded):
        result[(float(lat), float(lon))] = weather_map[(rlat, rlon)]
    return result


def compute_weather_factor(desc: str, precip_mm_hr: float) -> float:
    d = (desc or "").lower()
    if precip_mm_hr and precip_mm_hr > 0:
        return 1.25
    if any(k in d for k in ["thunderstorm", "snow", "sleet"]):
        return 1.3
    if "rain" in d or "drizzle" in d:
        return 1.2
    if "mist" in d or "fog" in d or "haze" in d or "smoke" in d:
        return 1.1
    return 1.0


# ----------------- Utility functions -----------------

def haversine_km(lat1, lon1, lat2, lon2):
    # digit-by-digit haversine
    R = 6371.0
    lat1r, lon1r, lat2r, lon2r = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2r - lat1r
    dlon = lon2r - lon1r
    a = sin(dlat / 2) ** 2 + cos(lat1r) * cos(lat2r) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    return R * c


# ----------------- Streamlit app -----------------

st.set_page_config(layout="wide", page_title="Fleet assignments dashboard")

st.sidebar.title("Data & live options")
# UPDATED: default paths to the uploaded files
assignments_csv = st.sidebar.text_input(
    "Assignments CSV path",
    "/content/drive/MyDrive/assignments_log.csv"
)
fleet_csv = st.sidebar.text_input(
    "Fleet CSV path",
    "/content/drive/MyDrive/synthetic_fleet_with_coords (1).csv"
)
demand_csv = st.sidebar.text_input(
    "Demand CSV path",
    "/content/drive/MyDrive/synthetic_demand_data.csv"
)

poll_cycles = st.sidebar.number_input("TomTom/OpenWeather poll cycles (per refresh)", min_value=1, max_value=10, value=1, step=1)
poll_interval_sec = st.sidebar.number_input("Poll interval seconds (between cycles)", min_value=1, max_value=300, value=1, step=1)

st.sidebar.markdown("---")
st.sidebar.write("API keys read from environment variables:")
st.sidebar.write(f"TomTom key loaded: {'YES' if TOMTOM_API_KEY else 'NO'}")
st.sidebar.write(f"OpenWeather key loaded: {'YES' if OPENWEATHER_API_KEY else 'NO'}")

if st.sidebar.button("Refresh live traffic & weather now"):
    st.experimental_rerun()

@st.cache_data(ttl=30)
def load_csvs(a_path, f_path, d_path):
    a = pd.read_csv(a_path) if os.path.exists(a_path) else pd.DataFrame()
    f = pd.read_csv(f_path) if os.path.exists(f_path) else pd.DataFrame()
    d = pd.read_csv(d_path) if os.path.exists(d_path) else pd.DataFrame()
    return a, f, d


assn_df, fleet_df, demand_df = load_csvs(assignments_csv, fleet_csv, demand_csv)

if assn_df.empty:
    st.warning("Assignments CSV not found or empty. Provide assignments.csv path in sidebar.")
    st.stop()

# prepare joined map DataFrame
df_map = assn_df.copy()

# attempt to merge lat/lon for demands and fleet if those are in separate files
# We assume demand_df has columns: d_idx (or index), lat, lon
# and fleet_df has columns: v_idx (or index), lat, lon
if ('lat' not in df_map.columns or 'lon' not in df_map.columns) and (not demand_df.empty):
    # try different possible column naming
    if 'd_idx' in df_map.columns:
        if 'd_idx' in demand_df.columns and {'lat', 'lon'}.issubset(demand_df.columns):
            df_map = df_map.merge(demand_df[['d_idx', 'lat', 'lon']], on='d_idx', how='left')
            df_map = df_map.rename(columns={'lat': 'd_lat', 'lon': 'd_lon'})
        elif {'lat', 'lon'}.issubset(demand_df.columns):
            # assume index alignment
            demand_df = demand_df.reset_index().rename(columns={'index': 'd_idx'})
            df_map = df_map.merge(demand_df[['d_idx', 'lat', 'lon']], on='d_idx', how='left')
            df_map = df_map.rename(columns={'lat': 'd_lat', 'lon': 'd_lon'})

if fleet_df is not None and not fleet_df.empty and 'v_idx' in df_map.columns:
    if 'v_idx' in fleet_df.columns and {'lat', 'lon'}.issubset(fleet_df.columns):
        fleet_coords = fleet_df[['v_idx', 'lat', 'lon']].rename(columns={'lat': 'v_lat', 'lon': 'v_lon'})
        df_map = df_map.merge(fleet_coords, on='v_idx', how='left')
    else:
        # try index alignment
        ff = fleet_df.reset_index().rename(columns={'index': 'v_idx'})
        if {'v_idx', 'lat', 'lon'}.issubset(ff.columns):
            ff = ff[['v_idx', 'lat', 'lon']].rename(columns={'lat': 'v_lat', 'lon': 'v_lon'})
            df_map = df_map.merge(ff, on='v_idx', how='left')

# fallback: if assn_df already had lat/lon for demand as 'lat'/'lon', copy to d_lat/d_lon
if 'lat' in assn_df.columns and 'd_lat' not in df_map.columns:
    df_map['d_lat'] = df_map['lat']
    df_map['d_lon'] = df_map['lon']

# center for maps
if 'd_lat' in df_map.columns and 'd_lon' in df_map.columns:
    center_lat = float(df_map['d_lat'].dropna().mean())
    center_lon = float(df_map['d_lon'].dropna().mean())
elif 'v_lat' in df_map.columns and 'v_lon' in df_map.columns:
    center_lat = float(df_map['v_lat'].dropna().mean())
    center_lon = float(df_map['v_lon'].dropna().mean())
else:
    center_lat, center_lon = 0.0, 0.0

# Top metrics
st.title("Fleet assignments dashboard")
c1, c2, c3, c4 = st.columns(4)
c1.metric("Total reward", round(assn_df['reward'].sum() if 'reward' in assn_df.columns else 0.0, 3))
c2.metric("Avg ETA (min)", round(assn_df['eta_min'].mean() if 'eta_min' in assn_df.columns else np.nan, 3))
c3.metric("Avg distance (km)", round(assn_df['dist_km'].mean() if 'dist_km' in assn_df.columns else np.nan, 3))
c4.metric("Assignments", len(assn_df))

tabs = st.tabs(["Map: assignments", "Capacity vs Demand", "Distance matrix", "Traffic (live)", "Weather (live)"])

# ---- Tab 1: assignments map ----
with tabs[0]:
    st.header("Assignments map (vehicle -> demand)")
    fol_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)
    for _, r in df_map.iterrows():
        lat_d = r.get('d_lat') or r.get('lat')
        lon_d = r.get('d_lon') or r.get('lon')
        lat_v = r.get('v_lat')
        lon_v = r.get('v_lon')
        popup_d = f"d_idx: {r.get('d_idx', '')}<br>size: {r.get('dsize', '')}"
        popup_v = f"v_idx: {r.get('v_idx', '')}<br>capacity: {r.get('capacity', '')}"
        try:
            if pd.notnull(lat_d) and pd.notnull(lon_d):
                folium.CircleMarker([lat_d, lon_d], radius=4, color='blue', fill=True, popup=popup_d).add_to(fol_map)
            if pd.notnull(lat_v) and pd.notnull(lon_v):
                folium.CircleMarker([lat_v, lon_v], radius=4, color='green', fill=True, popup=popup_v).add_to(fol_map)
            if pd.notnull(lat_d) and pd.notnull(lon_d) and pd.notnull(lat_v) and pd.notnull(lon_v):
                folium.PolyLine([[lat_v, lon_v], [lat_d, lon_d]], color='gray', weight=1).add_to(fol_map)
        except Exception:
            continue
    st_folium(fol_map, width=1000)

# ---- Tab 2: capacity vs demand ----
with tabs[1]:
    st.header("How demand size is satisfied by vehicle capacity")
    if {'dsize', 'capacity'}.issubset(assn_df.columns):
        assn = assn_df.copy()
        assn['utilization'] = assn['dsize'] / assn['capacity']
        fig_hist = px.histogram(assn, x='utilization', nbins=30, title="Distribution of utilization (dsize / capacity)")
        st.plotly_chart(fig_hist, use_container_width=True)

        agg = assn.groupby('v_idx', dropna=False).agg(total_demand=pd.NamedAgg('dsize', 'sum'),
                                                       capacity=pd.NamedAgg('capacity', 'first')).reset_index()
        agg['remaining'] = agg['capacity'] - agg['total_demand']
        fig_bar = px.bar(agg, x='v_idx', y=['total_demand', 'remaining'], title="Per vehicle: served vs remaining capacity")
        st.plotly_chart(fig_bar, use_container_width=True)
    else:
        st.info("Columns 'dsize' and 'capacity' required in assignments.csv to show capacity utilization.")

# ---- Tab 3: distance matrix / per-assignment distances ----
with tabs[2]:
    st.header("Distance between fleet and demand (per assignment) and pairwise fleet distances")
    dfd = df_map.copy()
    # ensure dlat/dlon and vlat/vlon exist
    dfd['dlat'] = dfd.get('d_lat', dfd.get('lat'))
    dfd['dlon'] = dfd.get('d_lon', dfd.get('lon'))
    dfd['vlat'] = dfd.get('v_lat')
    dfd['vlon'] = dfd.get('v_lon')
    if dfd[['dlat', 'dlon']].notna().any().any() and dfd[['vlat', 'vlon']].notna().any().any():
        dfd['dist_calc_km'] = dfd.apply(lambda r: haversine_km(r['vlat'], r['vlon'], r['dlat'], r['dlon']) if pd.notnull(r['vlat']) and pd.notnull(r['dlat']) else np.nan, axis=1)
        st.dataframe(dfd[['d_idx', 'v_idx', 'dist_km', 'dist_calc_km']].head(300))
        # pairwise fleet centroid distances
        try:
            fleet_centroids = dfd.groupby('v_idx').agg(vlat=('vlat','mean'), vlon=('vlon','mean')).dropna()
            idxs = list(fleet_centroids.index)
            mat = np.zeros((len(idxs), len(idxs)))
            for i, vi in enumerate(idxs):
                for j, vj in enumerate(idxs):
                    mat[i,j] = haversine_km(fleet_centroids.loc[vi,'vlat'], fleet_centroids.loc[vi,'vlon'],
                                            fleet_centroids.loc[vj,'vlat'], fleet_centroids.loc[vj,'vlon'])
            figm = px.imshow(mat, x=idxs, y=idxs, labels=dict(x="v_idx", y="v_idx", color="km"), title="Pairwise fleet centroid distances (km)")
            st.plotly_chart(figm, use_container_width=True)
        except Exception as e:
            st.write("Could not compute pairwise fleet matrix:", e)
    else:
        st.info("Need vehicle and demand lat/lon (v_lat/v_lon and d_lat/d_lon) to compute distances.")

# ---- helper: fetch live traffic & weather DataFrames ----

def fetch_live_traffic_from_tomtom(demand_points, poll_cycles_local=1, interval_sec_local=1):
    """
    demand_points: pandas DataFrame with columns 'dlat','d_lon' or 'lat','lon' (demand points).
    Returns DataFrame with columns lat, lon, traffic_speed_kmh
    """
    if demand_points is None or demand_points.empty:
        return pd.DataFrame(columns=['lat','lon','traffic_speed_kmh'])

    coords = []
    if {'d_lat','d_lon'}.issubset(demand_points.columns):
        coords = [(float(r.d_lat), float(r.d_lon)) for _, r in demand_points.iterrows()]
    elif {'lat','lon'}.issubset(demand_points.columns):
        coords = [(float(r.lat), float(r.lon)) for _, r in demand_points.iterrows()]
    else:
        return pd.DataFrame(columns=['lat','lon','traffic_speed_kmh'])

    last_df = None
    for i in range(1, poll_cycles_local + 1):
        tt_speed_by_coord = batch_get_tomtom_speeds(coords, pause_sec=0.1)
        rows = []
        for (lat, lon) in coords:
            speed = tt_speed_by_coord.get((float(lat), float(lon)), None)
            rows.append({"lat": float(lat), "lon": float(lon), "traffic_speed_kmh": float(speed)})
        last_df = pd.DataFrame(rows)
        # small print/log
        st.info(f"[{datetime.datetime.now().isoformat()}] TomTom speeds fetched (cycle {i}/{poll_cycles_local}).")
        if i < poll_cycles_local:
            time.sleep(interval_sec_local)
    return last_df

def fetch_live_weather_from_openweather(demand_points, poll_cycles_local=1, interval_sec_local=1):
    """
    demand_points: pandas DataFrame with columns 'd_lat','d_lon' or 'lat','lon'
    Returns DataFrame with lat, lon, description, temp_c, humidity, wind_speed_ms, precip_mm_hr, weather_factor
    """
    if demand_points is None or demand_points.empty:
        return pd.DataFrame(columns=['lat','lon','description','temp_c','humidity','wind_speed_ms','precip_mm_hr','weather_factor'])

    coords = []
    if {'d_lat','d_lon'}.issubset(demand_points.columns):
        coords = [(float(r.d_lat), float(r.d_lon)) for _, r in demand_points.iterrows()]
    elif {'lat','lon'}.issubset(demand_points.columns):
        coords = [(float(r.lat), float(r.lon)) for _, r in demand_points.iterrows()]
    else:
        return pd.DataFrame(columns=['lat','lon','description','temp_c','humidity','wind_speed_ms','precip_mm_hr','weather_factor'])

    last_df = None
    for i in range(1, poll_cycles_local + 1):
        ow_by_coord = batch_get_openweather(coords, pause_sec=0.1)
        rows = []
        for (lat, lon) in coords:
            w = ow_by_coord.get((float(lat), float(lon)), {})
            desc = w.get('description', '')
            temp_c = w.get('temp_c', np.nan)
            humidity = w.get('humidity', np.nan)
            wind_ms = w.get('wind_speed_ms', np.nan)
            precip = w.get('precip_mm_hr', 0.0)
            factor = compute_weather_factor(desc, precip)
            rows.append({"lat": float(lat), "lon": float(lon),
                         "description": desc, "temp_c": temp_c, "humidity": humidity,
                         "wind_speed_ms": wind_ms, "precip_mm_hr": precip, "weather_factor": factor})
        last_df = pd.DataFrame(rows)
        st.info(f"[{datetime.datetime.now().isoformat()}] OpenWeather fetched (cycle {i}/{poll_cycles_local}).")
        if i < poll_cycles_local:
            time.sleep(interval_sec_local)
    return last_df

# ---- Tab 4: traffic (live) ----
with tabs[3]:
    st.header("Traffic speed plotted on demand locations (TomTom)")
    # choose demand sample points (use df_map)
    demand_points = df_map
    traffic_df = fetch_live_traffic_from_tomtom(demand_points, poll_cycles_local=poll_cycles, interval_sec_local=poll_interval_sec)
    if traffic_df is not None and not traffic_df.empty:
        # pydeck scatter
        try:
            view = pdk.ViewState(latitude=float(traffic_df['lat'].mean()), longitude=float(traffic_df['lon'].mean()), zoom=12)
            layer = pdk.Layer("ScatterplotLayer", data=traffic_df, get_position='[lon, lat]', get_radius=80,
                              get_fill_color='[255 - (traffic_speed_kmh*5), (traffic_speed_kmh*5), 60, 180]', pickable=True)
            deck = pdk.Deck(layers=[layer], initial_view_state=view, tooltip={"text":"Speed: {traffic_speed_kmh} km/h"})
            st.pydeck_chart(deck, use_container_width=True)
        except Exception:
            # fallback to folium
            fmap = folium.Map(location=[center_lat, center_lon], zoom_start=12)
            for _, r in traffic_df.iterrows():
                folium.CircleMarker([r.lat, r.lon], radius=6, popup=f"speed: {r.traffic_speed_kmh} km/h").add_to(fmap)
            st_folium(fmap, width=900)
        st.dataframe(traffic_df.head())
    else:
        st.info("No traffic results (TomTom key missing or no demand points).")

# ---- Tab 5: weather (live) ----
with tabs[4]:
    st.header("Weather plotted on demand locations (OpenWeather)")
    weather_df = fetch_live_weather_from_openweather(df_map, poll_cycles_local=poll_cycles, interval_sec_local=poll_interval_sec)
    if weather_df is not None and not weather_df.empty:
        fmap = folium.Map(location=[float(weather_df['lat'].mean()), float(weather_df['lon'].mean())], zoom_start=12)
        for _, r in weather_df.iterrows():
            popup = f"desc: {r.get('description')}<br>temp: {r.get('temp_c')}°C<br>precip: {r.get('precip_mm_hr')} mm/h"
            folium.CircleMarker([r.lat, r.lon], radius=6, popup=popup, fill=True).add_to(fmap)
        st_folium(fmap, width=1000)
        st.dataframe(weather_df.head())
    else:
        st.info("No weather results (OpenWeather key missing or no demand points).")

st.markdown("---")
st.caption("Notes: live TomTom/OpenWeather calls are rate-limited in the helper functions. Use poll_cycles & interval in the sidebar to control how many times and spacing between polls. If you don't set API keys, the app will simulate traffic/weather values.")
# Colab cell (python)
import time, os, signal, subprocess
# ensure previous log is removed for clarity
if os.path.exists("streamlit.log"):
    os.remove("streamlit.log")

# start streamlit in the background and redirect logs
cmd = "streamlit run streamlit_app.py --server.port 8501 --server.enableCORS false --server.headless true"
print("Launching Streamlit with command:", cmd)

# use nohup to keep process alive and write logs to streamlit.log
get_ipython().system_raw(f"nohup {cmd} > streamlit.log 2>&1 &")

# wait a moment for server to start
time.sleep(3)
print("Started Streamlit; waiting 3s then checking localhost:8501 ...")

# quick curl check (prints headers or 'connection refused')
#!curl -s -I http://localhost:8501 | sed -n '1,12p' || echo "curl failed to connect"
print("\nIf you see HTTP headers above (like HTTP/1.1 200 or 302), Streamlit is listening at localhost:8501.")
print("If curl failed or connection refused, open the streamlit.log file below to see errors.")
# show last lines of the log
print("\n--- streamlit.log (tail) ---")
#!tail -n 60 streamlit.log || true


Overwriting streamlit_app.py


In [12]:
from pyngrok import ngrok
import time
NGROK_AUTH_TOKEN = "32FvHNzjMh8tcnakAfPbw8PBn3D_6dJdLXgx91REcwmJPRFoM"   # <-- paste your token here (keep secret)
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# ensure no previous tunnels
ngrok.kill()
time.sleep(0.5)

# 4) Launch streamlit in background and create public URL
get_ipython().system_raw('streamlit run streamlit_app.py --server.port 8501 &')
public_url = ngrok.connect(8501, bind_tls=True)
print("Streamlit is running at:", public_url)

Streamlit is running at: NgrokTunnel: "https://cd8e52fc002a.ngrok-free.app" -> "http://localhost:8501"
